https://deap.readthedocs.io/en/master/tutorials/advanced/checkpoint.html

In [9]:
import pickle
import random
import numpy

from deap import creator, base, tools, algorithms

In [28]:
IND_SIZE = 10
NGEN = 1000
FREQ = 100
CXPB, MUTPB = 0.5, 0.2

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



def evaluate(individual):
    return sum(individual),


toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [42]:
def main(checkpoint=None):
    if checkpoint:
        with open(checkpoint, 'rb') as cp_file:
            cp = pickle.load(cp_file)
        population = cp['population']
        start_gen = cp['generation']
        halloffame = cp['halloffame']
        logbook = cp['logbook']
        random.setstate(cp['rndstate'])
    else:
        population = toolbox.population(n=300)
        start_gen = 0
        halloffame = tools.HallOfFame(maxsize=1)
        logbook = tools.Logbook()
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register('max', numpy.max)
    
    for gen in range(start_gen, NGEN):
        population = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
        
        invalid_ind = [ind for ind in population if not ind.fitness.valid]
        fitnesss = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesss):
            ind.fitness.values = fit
        
        halloffame.update(population)
        record = stats.compile(population)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        
        population = toolbox.select(population, k=len(population))
        
        if gen % FREQ == 0:
            cp = dict(population=population, generation=gen, halloffame=halloffame, 
                     logbook=logbook, rndstate=random.getstate())
            
            with open('checkpoint_name.pkl', 'wb') as cp_file:
                pickle.dump(cp, cp_file)


    print(logbook)
        

In [40]:
main()

avg      	evals	gen	max      
4.97519  	300  	0  	8.30802  
4.17498  	180  	1  	6.63423  
3.42105  	187  	2  	6.21327  
2.65249  	187  	3  	5.02555  
1.74359  	178  	4  	5.34088  
0.804682 	178  	5  	3.98114  
-0.201584	187  	6  	3.91426  
-1.34168 	179  	7  	1.92758  
-2.45032 	202  	8  	2.9822   
-3.96269 	180  	9  	1.82832  
-5.34545 	171  	10 	-1.31983 
-6.62626 	183  	11 	-0.868938
-7.93191 	185  	12 	-3.67978 
-9.30843 	176  	13 	-2.94952 
-10.8912 	163  	14 	-6.5962  
-12.3256 	181  	15 	-8.06403 
-13.762  	188  	16 	-6.99924 
-15.3226 	183  	17 	-11.1965 
-16.7384 	178  	18 	-11.6295 
-18.2203 	167  	19 	-10.9621 
-19.8184 	190  	20 	-14.5749 
-21.2701 	197  	21 	-14.8671 
-22.3926 	181  	22 	-17.888  
-23.6795 	179  	23 	-17.9655 
-25.3302 	156  	24 	-19.6881 
-27.1128 	186  	25 	-20.4774 
-28.9376 	173  	26 	-23.3238 
-30.3784 	174  	27 	-25.2467 
-31.8145 	174  	28 	-24.6294 
-33.7602 	194  	29 	-27.2007 
-35.5886 	177  	30 	-29.8289 
-37.028  	177  	31 	-30.8182 
-38.5324 	

In [47]:
NGEN += 10000
main('checkpoint_name.pkl')

avg      	evals	gen  	max      
4.97519  	300  	0    	8.30802  
4.17498  	180  	1    	6.63423  
3.42105  	187  	2    	6.21327  
2.65249  	187  	3    	5.02555  
1.74359  	178  	4    	5.34088  
0.804682 	178  	5    	3.98114  
-0.201584	187  	6    	3.91426  
-1.34168 	179  	7    	1.92758  
-2.45032 	202  	8    	2.9822   
-3.96269 	180  	9    	1.82832  
-5.34545 	171  	10   	-1.31983 
-6.62626 	183  	11   	-0.868938
-7.93191 	185  	12   	-3.67978 
-9.30843 	176  	13   	-2.94952 
-10.8912 	163  	14   	-6.5962  
-12.3256 	181  	15   	-8.06403 
-13.762  	188  	16   	-6.99924 
-15.3226 	183  	17   	-11.1965 
-16.7384 	178  	18   	-11.6295 
-18.2203 	167  	19   	-10.9621 
-19.8184 	190  	20   	-14.5749 
-21.2701 	197  	21   	-14.8671 
-22.3926 	181  	22   	-17.888  
-23.6795 	179  	23   	-17.9655 
-25.3302 	156  	24   	-19.6881 
-27.1128 	186  	25   	-20.4774 
-28.9376 	173  	26   	-23.3238 
-30.3784 	174  	27   	-25.2467 
-31.8145 	174  	28   	-24.6294 
-33.7602 	194  	29   	-27.2007 
-35.5886